In [1]:
## Libraries

import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from statsmodels.formula.api import ols
import statsmodels.api as sm
from numpy.linalg import inv, solve
from scipy.stats import f

In [3]:
## Data loading 

def load_data():
    vw_industries = pd.read_excel('../data/Problem_Set4.xlsx', sheet_name = 'Industry portfolios (VW)', header=2)
    vw_industries = vw_industries.loc[:, :'Other']
    pr_portfolios = pd.read_excel('../data/Problem_Set4.xlsx', sheet_name='Past return portfolios', header=8)
    beme_portfolios = pd.read_excel('../data/Problem_Set4.xlsx', sheet_name='25 size and BEME portfolios', header=[1, 2])
    factors = pd.read_excel('../data/Problem_Set4.xlsx', sheet_name='Market, Rf', header =1)
    
    new_cols = []
    for col in beme_portfolios.columns:
        if 'Unnamed' in str(col[0]):
            new_cols.append('Date')
        else:
            new_cols.append(f"{col[0]}_{col[1]}")
    beme_portfolios.columns = new_cols

    return vw_industries, pr_portfolios, beme_portfolios, factors

vw_industries, pr_portfolios, beme_portfolios ,factors = load_data()

# Clean the data: 
def clean_data(df, replacement):
    df = df.replace([-99.99, -999], replacement)
    df['date'] = pd.to_datetime(df.iloc[:,0], format='%Y%m')
    df.set_index('date', inplace=True)
    df = df.iloc[:,1:]
    return df

vw_industries = clean_data(vw_industries,  np.nan)
pr_portfolios = clean_data(pr_portfolios, np.nan)
beme_portfolios = clean_data(beme_portfolios, np.nan)
factors = clean_data(factors, np.nan)

def excess_returns(df, factors):
    df = df.subtract(factors['RF'], axis=0)
    return df

vw_industries = excess_returns(vw_industries, factors)
pr_portfolios = excess_returns(pr_portfolios, factors)
beme_portfolios = excess_returns(beme_portfolios, factors)

print(vw_industries.head(5), pr_portfolios.head(5), beme_portfolios.head(5), sep='\n\n')

            Food   Beer  Smoke  Games  Books  Hshld  Clths  Hlth  Chems  \
date                                                                      
1926-07-01  0.34  -5.41   1.07   2.71  10.75  -0.70   7.86  1.55   7.92   
1926-08-01  2.34  26.78   6.25   0.30   9.76  -3.83  -2.76  4.00   5.25   
1926-09-01  0.93   3.79   1.03   6.35  -1.22   0.50  -0.74  0.46   5.10   
1926-10-01 -3.38  -3.63   0.74  -5.08   9.15  -5.00  -0.20 -0.89  -5.08   
1926-11-01  6.04   6.98   4.24   1.35  -6.11  -0.85   1.56  5.11   4.89   

            Txtls  ...  Telcm  Servs  BusEq  Paper  Trans  Whlsl  Rtail  \
date               ...                                                    
1926-07-01   0.17  ...   0.61   9.00   1.84   7.48   1.71 -24.01  -0.15   
1926-08-01   7.89  ...   1.92   1.77   4.14  -2.63   4.63   5.14  -1.00   
1926-09-01   2.08  ...   2.18   2.02  -0.04  -5.77  -0.18  -8.10   0.02   
1926-10-01   0.68  ...  -0.43  -2.32  -1.41  -5.40  -2.96 -15.70  -2.52   
1926-11-01   2.80  ...  

In [4]:
'''
Notes on TA session: 
- Problem B, just follow the formulas and calculate the GRS statistic => GRS is testing whether a portfolio is mean variance efficient. Which means 
that there is no other portfolio that has a higher return for the same level of risk.
- Problem D => economic significance and statistical signifcance ("alpha of 5% but p values is very large 60%, so we reject the null hypothesis at 5% level but 
the magnitude is still very high"). Remember if alpha is positive then there some return that is not explained by the risk factors. 
- Part B, we just changed the dataset 
- Part C => "How is it different to last weeks assignment? 
    - 
'''

'\nNotes on TA session: \n- Problem B, just follow the formulas and calculate the GRS statistic => GRS is testing whether a portfolio is mean variance efficient. Which means \nthat there is no other portfolio that has a higher return for the same level of risk.\n- Problem D => economic significance and statistical signifcance ("alpha of 5% but p values is very large 60%, so we reject the null hypothesis at 5% level but \nthe magnitude is still very high"). Remember if alpha is positive then there some return that is not explained by the risk factors. \n- Part B, we just changed the dataset \n- Part C => "How is it different to last weeks assignment? \n    - \n'

In [4]:
### Part 1
## Question A 
''' Lets calculate the sample mean and sample standard deviation for each of the 30 portfolios.'''
results = []

def summary_stats(df):
    for columns in df.columns: 
        mean = df[columns].mean()
        std = df[columns].std()
        sharpe_ratio = mean / std if std != 0 else 0
        results.append((columns, mean, std, sharpe_ratio))
    df_results = pd.DataFrame(results, columns= ['Portfolio', 'Mean', 'Std', 'Sharpe Ratio'])
    df_results.set_index('Portfolio', inplace=True)
    overall_mean = df_results['Mean'].mean()
    overall_std = df_results['Std'].mean()
    overall_sharpe = overall_mean / overall_std if overall_std != 0 else 0
    df_results.loc['Overall'] = [overall_mean, overall_std, overall_sharpe]
    return df_results

summary_stats = summary_stats(vw_industries)


In [5]:
## Question B 

def estimate_regressions(x, y):
    results = []
    # **FIXED:** Use y.columns instead of the hardcoded vw_industries.columns 
    # to keep the function general if you reuse it.
    for columns in y.columns:
        X = sm.add_constant(x)
        Y = y[columns]
        # Data alignment should be handled in the call, but OLS handles indices automatically
        model =sm.OLS(Y, X, missing='drop').fit()
        alpha = model.params['const']
        beta = model.params[x.columns[0]]
        residuals = model.resid
        results.append((columns, alpha, beta, residuals))
    return pd.DataFrame(results, columns=['Portfolio', 'Alpha', 'Beta', 'Residuals']).set_index('Portfolio')

# Use vw_industries (already excess returns) as the test assets (y)
regression_results = estimate_regressions(factors[['RM-RF']], vw_industries)
print(regression_results)

# Careful need to check if this covariance matrix is unbiased or not 
def residuals_covariance_matrix(residuals):
    # Stack residuals into a 2D array: shape (n_portfolios, n_periods) of dimension (n_portfolios x n_periods)
    residuals_matrix = np.vstack([r.values for r in residuals.values]).T 
    # **FIXED:** Changed to rowvar=False for T rows, N columns to ensure correct N x N covariance 
    # calculation, which is the preferred way when data is organized T x N.
    residuals_covariance_matrix = np.cov(residuals_matrix, rowvar=False)
    return residuals_covariance_matrix

sigma_hat = residuals_covariance_matrix(regression_results['Residuals'])

# Now lets compute the sample means and sample covariance matrix 
def sample_means_and_covariance(returns):
    returns_matrix = returns.values
    # Calculates mean for each column (factor)
    sample_means_vector = returns_matrix.mean(axis=0) 
    # Computes K x K covariance matrix (rowvar=False assumes T rows, K columns)
    sample_means_covariance = np.cov(returns_matrix, rowvar=False)
    return sample_means_vector, sample_means_covariance

# **CORRECTION:** The call uses the correct factor data (RM-RF)
sample_means, sample_covariance_matrix = sample_means_and_covariance(factors[['RM-RF']])
print(sample_means, sample_covariance_matrix)

def grs_statistic(T, N, K, alpha_vector, sigma_hat, m_hat, omega_hat):
    """
    Computes the GRS test statistic. (Fixed scaling and input reshaping)

    Args:
        T (int): Number of time periods.
        N (int): Number of test assets/portfolios.
        K (int): Number of factors.
        alpha_vector (np.ndarray): N x 1 vector of estimated intercepts (alphas).
        sigma_hat (np.ndarray): N x N covariance matrix of residuals (Sigma).
        m_hat (np.ndarray): K x 1 vector of factor means (m).
        omega_hat (np.ndarray): K x K covariance matrix of factors (Omega).

    Returns:
        float: The GRS test statistic.
    """
    
    # Ensure inputs are correctly shaped for matrix algebra
    alpha_vector = alpha_vector.reshape(-1, 1)
    m_hat = m_hat.reshape(-1, 1)
    # If omega_hat is a scalar (K=1), ensure it's treated as a 1x1 array
    if K == 1 and omega_hat.ndim == 0:
        omega_hat = np.array([[omega_hat]])

    # 1. Numerator: alpha' * Sigma_inv * alpha
    numerator_quadratic = alpha_vector.T @ solve(sigma_hat, alpha_vector)
    
    # 2. Denominator: 1 + m' * Omega_inv * m
    denominator_quadratic = m_hat.T @ solve(omega_hat, m_hat)
    denominator = 1 + denominator_quadratic

    # GRS Statistic (W)
    grs_stat = (numerator_quadratic / denominator)
    grs_stat_w = grs_stat[0][0]

    # 3. Scaling factor (FIXED TO STANDARD GRS F-STATISTIC FORMULA)
    df1 = N
    df2 = T - N - K
    # Standard GRS F-statistic scaling for F(N, T-N-K) distribution
    scaling_factor_f = (T - N - K) / N
    
    grs_stat_normalised = grs_stat_w * scaling_factor_f

    # 4. Compute the p-value
    p_value = f.sf(grs_stat_normalised, df1, df2)

    return grs_stat_w, grs_stat_normalised, p_value

# Determine K based on the factor data used in the regression call
K = factors[['RM-RF']].shape[1] 

# **CORRECTION:** Reshape K-dimensional factor moments to be safe for the function call
m_hat_reshaped = sample_means.reshape(K, 1)
omega_hat_reshaped = sample_covariance_matrix.reshape(K, K)

# The shape attributes (T, N, K) are calculated correctly in Cell 1/start of Cell 3.
grs_stat_w, grs_stat_f, p_value = grs_statistic(
    T=vw_industries.shape[0], 
    N=vw_industries.shape[1], 
    K=K, 
    alpha_vector=regression_results['Alpha'].values, 
    sigma_hat=sigma_hat, # Now based on correct residuals covariance calculation
    m_hat=m_hat_reshaped, 
    omega_hat=omega_hat_reshaped
)

print(f"GRS Statistic (W): {grs_stat_w}, Scaled GRS F-Statistic: {grs_stat_f}, p-value: {p_value}")

''' basically the hypothesis test in a GRS tests to see if the alphas are jointly equal to zero. 
Jointly equal to zero means that there is no portfolio that can beat the market portfolio.'''

              Alpha      Beta  \
Portfolio                       
Food       0.218167  0.739370   
Beer       0.317082  0.942062   
Smoke      0.482244  0.628665   
Games     -0.059115  1.388648   
Books     -0.076998  1.107988   
Hshld      0.057837  0.902271   
Clths      0.117064  0.813177   
Hlth       0.255263  0.839263   
Chems      0.103032  1.042579   
Txtls     -0.017285  1.138909   
Cnstr     -0.102418  1.172375   
Steel     -0.243096  1.357006   
FabPr     -0.035984  1.241626   
ElcEq      0.057924  1.284064   
Autos     -0.012242  1.252084   
Carry      0.073151  1.188701   
Mines      0.042680  0.908179   
Coal      -0.050477  1.299820   
Oil        0.185034  0.868141   
Util       0.093951  0.778292   
Telcm      0.153157  0.661480   
Servs      0.397597  0.812840   
BusEq      0.135660  1.077502   
Paper      0.132112  0.954503   
Trans     -0.091217  1.137771   
Whlsl     -0.158732  1.089085   
Rtail      0.120500  0.965016   
Meals      0.163777  0.945508   
Fin       

' basically the hypothesis test in a GRS tests to see if the alphas are jointly equal to zero. \nJointly equal to zero means that there is no portfolio that can beat the market portfolio.'

In [6]:
### Part 2
## Question A 
''' Lets calculate the sample mean and sample standard deviation for each of the 10 portfolios.'''
results = []

def summary_stats(df):
    for columns in df.columns: 
        mean = df[columns].mean()
        std = df[columns].std()
        sharpe_ratio = mean / std if std != 0 else 0
        results.append((columns, mean, std, sharpe_ratio))
    df_results = pd.DataFrame(results, columns= ['Portfolio', 'Mean', 'Std', 'Sharpe Ratio'])
    df_results.set_index('Portfolio', inplace=True)
    overall_mean = df_results['Mean'].mean()
    overall_std = df_results['Std'].mean()
    overall_sharpe = overall_mean / overall_std if overall_std != 0 else 0
    df_results.loc['Overall'] = [overall_mean, overall_std, overall_sharpe]
    return df_results

summary_stats_pr = summary_stats(pr_portfolios)
summary_stats_pr

,Mean,Std,Sharpe Ratio
Portfolio,,,
Loser,0.040287,9.807258,0.004108
2,0.420964,8.065572,0.052193
3,0.474041,6.995907,0.067760
4,0.599333,6.366311,0.094141
5,0.604365,5.964666,0.101324
6,0.657665,5.814916,0.113100
7,0.736172,5.508820,0.133635
8,0.839082,5.352057,0.156778
9,0.920176,5.635042,0.163295


In [7]:
## Question B 

def estimate_regressions(x, y):
    results = []
    # **FIXED:** Use y.columns instead of the hardcoded pr_portfolios.columns 
    # to keep the function general if you reuse it.
    for columns in y.columns:
        X = sm.add_constant(x)
        Y = y[columns]
        # Data alignment should be handled in the call, but OLS handles indices automatically
        model =sm.OLS(Y, X, missing='drop').fit()
        alpha = model.params['const']
        beta = model.params[x.columns[0]]
        residuals = model.resid
        results.append((columns, alpha, beta, residuals))
    return pd.DataFrame(results, columns=['Portfolio', 'Alpha', 'Beta', 'Residuals']).set_index('Portfolio')

# Use pr_portfolios (already excess returns) as the test assets (y)
regression_results = estimate_regressions(factors[['RM-RF']], pr_portfolios)
print(regression_results)

# Careful need to check if this covariance matrix is unbiased or not 
def residuals_covariance_matrix(residuals):
    # Stack residuals into a 2D array: shape (n_portfolios, n_periods) of dimension (n_portfolios x n_periods)
    residuals_matrix = np.vstack([r.values for r in residuals.values]).T 
    # **FIXED:** Changed to rowvar=False for T rows, N columns to ensure correct N x N covariance 
    # calculation, which is the preferred way when data is organized T x N.
    residuals_covariance_matrix = np.cov(residuals_matrix, rowvar=False)
    return residuals_covariance_matrix

sigma_hat = residuals_covariance_matrix(regression_results['Residuals'])

# Now lets compute the sample means and sample covariance matrix 
def sample_means_and_covariance(returns):
    returns_matrix = returns.values
    # Calculates mean for each column (factor)
    sample_means_vector = returns_matrix.mean(axis=0) 
    # Computes K x K covariance matrix (rowvar=False assumes T rows, K columns)
    sample_means_covariance = np.cov(returns_matrix, rowvar=False)
    return sample_means_vector, sample_means_covariance

# **CORRECTION:** The call uses the correct factor data (RM-RF)
sample_means, sample_covariance_matrix = sample_means_and_covariance(factors[['RM-RF']])
print(sample_means, sample_covariance_matrix)

def grs_statistic(T, N, K, alpha_vector, sigma_hat, m_hat, omega_hat):
    """
    Computes the GRS test statistic. (Fixed scaling and input reshaping)

    Args:
        T (int): Number of time periods.
        N (int): Number of test assets/portfolios.
        K (int): Number of factors.
        alpha_vector (np.ndarray): N x 1 vector of estimated intercepts (alphas).
        sigma_hat (np.ndarray): N x N covariance matrix of residuals (Sigma).
        m_hat (np.ndarray): K x 1 vector of factor means (m).
        omega_hat (np.ndarray): K x K covariance matrix of factors (Omega).

    Returns:
        float: The GRS test statistic.
    """
    
    # Ensure inputs are correctly shaped for matrix algebra
    alpha_vector = alpha_vector.reshape(-1, 1)
    m_hat = m_hat.reshape(-1, 1)
    # If omega_hat is a scalar (K=1), ensure it's treated as a 1x1 array
    if K == 1 and omega_hat.ndim == 0:
        omega_hat = np.array([[omega_hat]])

    # 1. Numerator: alpha' * Sigma_inv * alpha
    numerator_quadratic = alpha_vector.T @ solve(sigma_hat, alpha_vector)
    
    # 2. Denominator: 1 + m' * Omega_inv * m
    denominator_quadratic = m_hat.T @ solve(omega_hat, m_hat)
    denominator = 1 + denominator_quadratic

    # GRS Statistic (W)
    grs_stat = (numerator_quadratic / denominator)
    grs_stat_w = grs_stat[0][0]

    # 3. Scaling factor (FIXED TO STANDARD GRS F-STATISTIC FORMULA)
    df1 = N
    df2 = T - N - K
    # Standard GRS F-statistic scaling for F(N, T-N-K) distribution
    scaling_factor_f = (T - N - K) / N
    
    grs_stat_normalised = grs_stat_w * scaling_factor_f

    # 4. Compute the p-value
    p_value = f.sf(grs_stat_normalised, df1, df2)

    return grs_stat_w, grs_stat_normalised, p_value

# Determine K based on the factor data used in the regression call
K = factors[['RM-RF']].shape[1] 

# **CORRECTION:** Reshape K-dimensional factor moments to be safe for the function call
m_hat_reshaped = sample_means.reshape(K, 1)
omega_hat_reshaped = sample_covariance_matrix.reshape(K, K)

# The shape attributes (T, N, K) are calculated correctly in Cell 1/start of Cell 3.
grs_stat_w, grs_stat_f, p_value = grs_statistic(
    T=pr_portfolios.shape[0], 
    N=pr_portfolios.shape[1], 
    K=K, 
    alpha_vector=regression_results['Alpha'].values, 
    sigma_hat=sigma_hat, # Now based on correct residuals covariance calculation
    m_hat=m_hat_reshaped, 
    omega_hat=omega_hat_reshaped
)

print(f"GRS Statistic (W): {grs_stat_w}, Scaled GRS F-Statistic: {grs_stat_f}, p-value: {p_value}")

''' basically the hypothesis test in a GRS tests to see if the alphas are jointly equal to zero. 
Jointly equal to zero means that there is no portfolio that can beat the market portfolio.'''

              Alpha      Beta  \
Portfolio                       
Loser     -0.969041  1.558457   
2         -0.439353  1.328375   
3         -0.287373  1.175664   
4         -0.108870  1.093502   
5         -0.068067  1.038272   
6         -0.010003  1.030915   
7          0.108158  0.969687   
8          0.234110  0.934110   
9          0.295572  0.964422   
Winner     0.562602  1.023815   

                                                   Residuals  
Portfolio                                                     
Loser      date
1927-01-01   -2.507452
1927-02-01    1.68...  
2          date
1927-01-01   -4.350945
1927-02-01    0.63...  
3          date
1927-01-01    2.767913
1927-02-01    3.44...  
4          date
1927-01-01   -0.395520
1927-02-01    2.78...  
5          date
1927-01-01   -0.529636
1927-02-01   -0.73...  
6          date
1927-01-01    0.781858
1927-02-01   -0.39...  
7          date
1927-01-01    0.470023
1927-02-01   -1.61...  
8          date
1927-01-01   -0.0680

' basically the hypothesis test in a GRS tests to see if the alphas are jointly equal to zero. \nJointly equal to zero means that there is no portfolio that can beat the market portfolio.'

In [8]:
### Part 3
## Question A 
''' Lets calculate the sample mean and sample standard deviation for each of the 10 portfolios.'''
results = []

def summary_stats(df):
    for columns in df.columns: 
        mean = df[columns].mean()
        std = df[columns].std()
        sharpe_ratio = mean / std if std != 0 else 0
        results.append((columns, mean, std, sharpe_ratio))
    df_results = pd.DataFrame(results, columns= ['Portfolio', 'Mean', 'Std', 'Sharpe Ratio'])
    df_results.set_index('Portfolio', inplace=True)
    overall_mean = df_results['Mean'].mean()
    overall_std = df_results['Std'].mean()
    overall_sharpe = overall_mean / overall_std if overall_std != 0 else 0
    df_results.loc['Overall'] = [overall_mean, overall_std, overall_sharpe]
    return df_results

summary_stats_beme = summary_stats(beme_portfolios)
summary_stats_beme

,Mean,Std,Sharpe Ratio
Portfolio,,,
Small_Low,0.563687,12.266929,0.045952
Small_2,0.691816,9.873443,0.070068
Small_3,0.994387,9.026214,0.110167
Small_4,1.183364,8.366578,0.141439
Small_High,1.364378,9.319727,0.146397
2_Low,0.621069,8.011713,0.077520
2_2,0.926046,7.519225,0.123157
2_3,0.990949,7.278665,0.136144
2_4,1.079475,7.444352,0.145006


In [9]:
## Question B 

def estimate_regressions(x, y):
    results = []
    # **FIXED:** Use y.columns instead of the hardcoded beme_portfolios.columns 
    # to keep the function general if you reuse it.
    for columns in y.columns:
        X = sm.add_constant(x)
        Y = y[columns]
        # Data alignment should be handled in the call, but OLS handles indices automatically
        model =sm.OLS(Y, X, missing='drop').fit()
        alpha = model.params['const']
        beta = model.params[x.columns[0]]
        residuals = model.resid
        results.append((columns, alpha, beta, residuals))
    return pd.DataFrame(results, columns=['Portfolio', 'Alpha', 'Beta', 'Residuals']).set_index('Portfolio')

# Use beme_portfolios (already excess returns) as the test assets (y)
regression_results = estimate_regressions(factors[['RM-RF']], beme_portfolios)
print(regression_results)

# Careful need to check if this covariance matrix is unbiased or not 
def residuals_covariance_matrix(residuals):
    # Stack residuals into a 2D array: shape (n_portfolios, n_periods) of dimension (n_portfolios x n_periods)
    residuals_matrix = np.vstack([r.values for r in residuals.values]).T 
    # **FIXED:** Changed to rowvar=False for T rows, N columns to ensure correct N x N covariance 
    # calculation, which is the preferred way when data is organized T x N.
    residuals_covariance_matrix = np.cov(residuals_matrix, rowvar=False)
    return residuals_covariance_matrix

sigma_hat = residuals_covariance_matrix(regression_results['Residuals'])

# Now lets compute the sample means and sample covariance matrix 
def sample_means_and_covariance(returns):
    returns_matrix = returns.values
    # Calculates mean for each column (factor)
    sample_means_vector = returns_matrix.mean(axis=0) 
    # Computes K x K covariance matrix (rowvar=False assumes T rows, K columns)
    sample_means_covariance = np.cov(returns_matrix, rowvar=False)
    return sample_means_vector, sample_means_covariance

# **CORRECTION:** The call uses the correct factor data (RM-RF)
sample_means, sample_covariance_matrix = sample_means_and_covariance(factors[['RM-RF']])
print(sample_means, sample_covariance_matrix)

def grs_statistic(T, N, K, alpha_vector, sigma_hat, m_hat, omega_hat):
    """
    Computes the GRS test statistic. (Fixed scaling and input reshaping)

    Args:
        T (int): Number of time periods.
        N (int): Number of test assets/portfolios.
        K (int): Number of factors.
        alpha_vector (np.ndarray): N x 1 vector of estimated intercepts (alphas).
        sigma_hat (np.ndarray): N x N covariance matrix of residuals (Sigma).
        m_hat (np.ndarray): K x 1 vector of factor means (m).
        omega_hat (np.ndarray): K x K covariance matrix of factors (Omega).

    Returns:
        float: The GRS test statistic.
    """
    
    # Ensure inputs are correctly shaped for matrix algebra
    alpha_vector = alpha_vector.reshape(-1, 1)
    m_hat = m_hat.reshape(-1, 1)
    # If omega_hat is a scalar (K=1), ensure it's treated as a 1x1 array
    if K == 1 and omega_hat.ndim == 0:
        omega_hat = np.array([[omega_hat]])

    # 1. Numerator: alpha' * Sigma_inv * alpha
    numerator_quadratic = alpha_vector.T @ solve(sigma_hat, alpha_vector)
    
    # 2. Denominator: 1 + m' * Omega_inv * m
    denominator_quadratic = m_hat.T @ solve(omega_hat, m_hat)
    denominator = 1 + denominator_quadratic

    # GRS Statistic (W)
    grs_stat = (numerator_quadratic / denominator)
    grs_stat_w = grs_stat[0][0]

    # 3. Scaling factor (FIXED TO STANDARD GRS F-STATISTIC FORMULA)
    df1 = N
    df2 = T - N - K
    # Standard GRS F-statistic scaling for F(N, T-N-K) distribution
    scaling_factor_f = (T - N - K) / N
    
    grs_stat_normalised = grs_stat_w * scaling_factor_f

    # 4. Compute the p-value
    p_value = f.sf(grs_stat_normalised, df1, df2)

    return grs_stat_w, grs_stat_normalised, p_value

# Determine K based on the factor data used in the regression call
K = factors[['RM-RF']].shape[1] 

# **CORRECTION:** Reshape K-dimensional factor moments to be safe for the function call
m_hat_reshaped = sample_means.reshape(K, 1)
omega_hat_reshaped = sample_covariance_matrix.reshape(K, K)

# The shape attributes (T, N, K) are calculated correctly in Cell 1/start of Cell 3.
grs_stat_w, grs_stat_f, p_value = grs_statistic(
    T=beme_portfolios.shape[0], 
    N=beme_portfolios.shape[1], 
    K=K, 
    alpha_vector=regression_results['Alpha'].values, 
    sigma_hat=sigma_hat, # Now based on correct residuals covariance calculation
    m_hat=m_hat_reshaped, 
    omega_hat=omega_hat_reshaped
)

print(f"GRS Statistic (W): {grs_stat_w}, Scaled GRS F-Statistic: {grs_stat_f}, p-value: {p_value}")

''' basically the hypothesis test in a GRS tests to see if the alphas are jointly equal to zero. 
Jointly equal to zero means that there is no portfolio that can beat the market portfolio.'''

               Alpha      Beta  \
Portfolio                        
Small_Low  -0.498134  1.630265   
Small_2    -0.225871  1.408970   
Small_3     0.100179  1.372921   
Small_4     0.356579  1.269402   
Small_High  0.465675  1.379822   
2_Low      -0.203407  1.265858   
2_2         0.127288  1.226372   
2_3         0.210850  1.197724   
2_4         0.289742  1.212515   
2_High      0.358482  1.378654   
3_Low      -0.112276  1.245103   
3_2         0.173301  1.125557   
3_3         0.201535  1.123797   
3_4         0.263645  1.159598   
3_High      0.260708  1.377896   
4_Low      -0.000194  1.091938   
4_2         0.044986  1.079654   
4_3         0.135973  1.115270   
4_4         0.218534  1.154101   
4_High      0.108564  1.420400   
Big_Low    -0.011064  0.955319   
Big_2       0.007790  0.949832   
Big_3       0.071279  0.968363   
Big_4      -0.070742  1.108444   
Big_High    0.105792  1.311918   

                                                    Residuals  
Portfolio        

' basically the hypothesis test in a GRS tests to see if the alphas are jointly equal to zero. \nJointly equal to zero means that there is no portfolio that can beat the market portfolio.'

In [10]:
## Question G
import numpy as np

# Step 1: Extract returns matrix
R = beme_portfolios.values    # T x N, here N=25
mu = R.mean(axis=0)           # vector of means (N,)
Sigma = np.cov(R, rowvar=False, ddof=1)  # N x N covariance

from numpy.linalg import inv

# Step 2: Tangency portfolio weights
ones = np.ones(mu.shape[0])
Sigma_inv = inv(Sigma)
w_unnormalized = Sigma_inv @ mu
w_tan = w_unnormalized / (ones @ w_unnormalized)

print("Tangency portfolio weights:", w_tan)
print("Sum of weights:", w_tan.sum())  # should be 1

# Step 3: Time-series of tangency portfolio returns
R_tan = R @ w_tan   # T x 1 vector
beme_portfolios['Tangency'] = R_tan


Tangency portfolio weights: [-0.45361504 -0.75250462 -0.47075843  0.81369172  0.88441869 -0.92607059
  0.0870831   0.32691723  0.7300126   0.31204972 -0.54907905  0.99560097
  0.35233356  0.36432601  0.13815683  0.97184336 -1.34094407 -0.10978097
  0.47690186 -0.72340751  0.70746216 -0.14676221  0.71140198 -1.4998225
  0.10054523]
Sum of weights: 0.9999999999999998


In [11]:
def tangency_factor_from_excess(df_excess: pd.DataFrame):
    """
    df_excess: T x N DataFrame of *excess* returns (e.g., your beme_portfolios after subtracting RF).
    Returns:
        w_tan : (N,) ndarray of tangency weights (sum to 1)
        R_tan : (T,) pandas Series of tangency (excess) returns
    """
    # Drop rows with any NaNs to keep moments well-defined (optional: use a stricter alignment if needed)
    R = df_excess.dropna(how='any').copy()
    mu = R.mean(axis=0).values                # (N,)
    Sigma = np.cov(R.values, rowvar=False, ddof=1)   # (N x N)

    # Unconstrained tangency weights (shorting allowed)
    # w* ∝ Σ^{-1} μ, then normalize so that 1'w = 1
    Sigma_inv_mu = solve(Sigma, mu)           # more stable than inv(Sigma) @ mu
    denom = np.sum(Sigma_inv_mu)
    w_tan = Sigma_inv_mu / denom              # (N,)

    # Tangency returns time series: R_tan_t = w' R_t
    R_tan_vals = R.values @ w_tan             # (T,)
    R_tan = pd.Series(R_tan_vals, index=R.index, name='R_tan')

    return w_tan, R_tan

# --- Use it with your data (already excess returns) ---
w_tan, R_tan = tangency_factor_from_excess(beme_portfolios)

# If you want a factor DataFrame for regressions:
tangency_factor = pd.DataFrame({'R_tan': R_tan})
tangency_factor

,R_tan
date,
1926-07-01,-2.980133
1926-08-01,9.036278
1926-09-01,6.241085
1926-10-01,10.379636
1926-11-01,-3.793419
...,...
2016-07-01,2.420384
2016-08-01,4.563088
2016-09-01,-3.139405


In [12]:
## Question G

def estimate_regressions(x, y):
    results = []
    # **FIXED:** Use y.columns instead of the hardcoded beme_portfolios.columns 
    # to keep the function general if you reuse it.
    for columns in y.columns:
        X = sm.add_constant(x)
        Y = y[columns]
        # Data alignment should be handled in the call, but OLS handles indices automatically
        model =sm.OLS(Y, X, missing='drop').fit()
        alpha = model.params['const']
        beta = model.params[x.columns[0]]
        residuals = model.resid
        results.append((columns, alpha, beta, residuals))
    return pd.DataFrame(results, columns=['Portfolio', 'Alpha', 'Beta', 'Residuals']).set_index('Portfolio')

# Use beme_portfolios (already excess returns) as the test assets (y)
regression_results = estimate_regressions(tangency_factor, beme_portfolios)
print(regression_results)

# Careful need to check if this covariance matrix is unbiased or not 
def residuals_covariance_matrix(residuals):
    # Stack residuals into a 2D array: shape (n_portfolios, n_periods) of dimension (n_portfolios x n_periods)
    residuals_matrix = np.vstack([r.values for r in residuals.values]).T 
    # **FIXED:** Changed to rowvar=False for T rows, N columns to ensure correct N x N covariance 
    # calculation, which is the preferred way when data is organized T x N.
    residuals_covariance_matrix = np.cov(residuals_matrix, rowvar=False)
    return residuals_covariance_matrix

sigma_hat = residuals_covariance_matrix(regression_results['Residuals'])

# Now lets compute the sample means and sample covariance matrix 
def sample_means_and_covariance(returns):
    returns_matrix = returns.values
    # Calculates mean for each column (factor)
    sample_means_vector = returns_matrix.mean(axis=0) 
    # Computes K x K covariance matrix (rowvar=False assumes T rows, K columns)
    sample_means_covariance = np.cov(returns_matrix, rowvar=False)
    return sample_means_vector, sample_means_covariance

# **CORRECTION:** The call uses the correct factor data (RM-RF)
sample_means, sample_covariance_matrix = sample_means_and_covariance(tangency_factor)
print(sample_means, sample_covariance_matrix)

def grs_statistic(T, N, K, alpha_vector, sigma_hat, m_hat, omega_hat):
    """
    Computes the GRS test statistic. (Fixed scaling and input reshaping)

    Args:
        T (int): Number of time periods.
        N (int): Number of test assets/portfolios.
        K (int): Number of factors.
        alpha_vector (np.ndarray): N x 1 vector of estimated intercepts (alphas).
        sigma_hat (np.ndarray): N x N covariance matrix of residuals (Sigma).
        m_hat (np.ndarray): K x 1 vector of factor means (m).
        omega_hat (np.ndarray): K x K covariance matrix of factors (Omega).

    Returns:
        float: The GRS test statistic.
    """
    
    # Ensure inputs are correctly shaped for matrix algebra
    alpha_vector = alpha_vector.reshape(-1, 1)
    m_hat = m_hat.reshape(-1, 1)
    # If omega_hat is a scalar (K=1), ensure it's treated as a 1x1 array
    if K == 1 and omega_hat.ndim == 0:
        omega_hat = np.array([[omega_hat]])

    # 1. Numerator: alpha' * Sigma_inv * alpha
    numerator_quadratic = alpha_vector.T @ solve(sigma_hat, alpha_vector)
    
    # 2. Denominator: 1 + m' * Omega_inv * m
    denominator_quadratic = m_hat.T @ solve(omega_hat, m_hat)
    denominator = 1 + denominator_quadratic

    # GRS Statistic (W)
    grs_stat = (numerator_quadratic / denominator)
    grs_stat_w = grs_stat[0][0]

    # 3. Scaling factor (FIXED TO STANDARD GRS F-STATISTIC FORMULA)
    df1 = N
    df2 = T - N - K
    # Standard GRS F-statistic scaling for F(N, T-N-K) distribution
    scaling_factor_f = (T - N - K) / N
    
    grs_stat_normalised = grs_stat_w * scaling_factor_f

    # 4. Compute the p-value
    p_value = f.sf(grs_stat_normalised, df1, df2)

    return grs_stat_w, grs_stat_normalised, p_value

# Determine K based on the factor data used in the regression call
K = tangency_factor.shape[1] 

# **CORRECTION:** Reshape K-dimensional factor moments to be safe for the function call
m_hat_reshaped = sample_means.reshape(K, 1)
omega_hat_reshaped = sample_covariance_matrix.reshape(K, K)

# The shape attributes (T, N, K) are calculated correctly in Cell 1/start of Cell 3.
grs_stat_w, grs_stat_f, p_value = grs_statistic(
    T=beme_portfolios.shape[0], 
    N=beme_portfolios.shape[1], 
    K=K, 
    alpha_vector=regression_results['Alpha'].values, 
    sigma_hat=sigma_hat, # Now based on correct residuals covariance calculation
    m_hat=m_hat_reshaped, 
    omega_hat=omega_hat_reshaped
)

print(f"GRS Statistic (W): {grs_stat_w}, Scaled GRS F-Statistic: {grs_stat_f}, p-value: {p_value}")

''' basically the hypothesis test in a GRS tests to see if the alphas are jointly equal to zero. 
Jointly equal to zero means that there is no portfolio that can beat the market portfolio.'''

                   Alpha      Beta  \
Portfolio                            
Small_Low  -1.824937e-15  0.218377   
Small_2    -1.834526e-15  0.268016   
Small_3    -2.487096e-15  0.385235   
Small_4    -4.048652e-15  0.458446   
Small_High -3.201939e-15  0.528572   
2_Low      -1.600737e-15  0.240608   
2_2        -2.968010e-15  0.358759   
2_3        -1.876356e-15  0.383903   
2_4        -2.804954e-15  0.418198   
2_High     -2.938578e-15  0.486750   
3_Low      -1.058572e-15  0.270676   
3_2        -3.118497e-15  0.351146   
3_3        -2.101209e-15  0.361640   
3_4        -3.412040e-15  0.394736   
3_High     -3.497262e-15  0.448681   
4_Low      -1.809675e-15  0.275450   
4_2        -2.224929e-15  0.289853   
4_3        -2.494938e-15  0.334090   
4_4        -3.143915e-15  0.375873   
4_High     -2.941337e-15  0.400463   
Big_Low    -2.777792e-15  0.236766   
Big_2      -2.024502e-15  0.242686   
Big_3      -2.026828e-15  0.271958   
Big_4      -2.160874e-15  0.252284   
Big_High   -

' basically the hypothesis test in a GRS tests to see if the alphas are jointly equal to zero. \nJointly equal to zero means that there is no portfolio that can beat the market portfolio.'

In [19]:
import numpy as np
import pandas as pd
from numpy.linalg import solve

# -------------------------------------------------------------------
# Helpers
# -------------------------------------------------------------------
def split_interleaved(df: pd.DataFrame):
    """Return two interleaved halves A and B as described."""
    idx = df.index
    even_year = (idx.year % 2 == 0)
    odd_year  = ~even_year
    even_mon  = (idx.month % 2 == 0)
    odd_mon   = ~even_mon

    # A: odd months in even years  OR  even months in odd years
    mask_A = (even_year & odd_mon) | (odd_year & even_mon)
    mask_B = ~mask_A

    A = df.loc[mask_A].dropna(how='any')
    B = df.loc[mask_B].dropna(how='any')
    return A, B

def tangency_weights(R_df: pd.DataFrame):
    """
    Unconstrained tangency (max Sharpe) weights using excess returns.
    w* ∝ Σ^{-1} μ, normalized so 1'w = 1.
    """
    mu = R_df.mean(axis=0).values                    # (N,)
    Sigma = np.cov(R_df.values, rowvar=False, ddof=1)  # (N x N)
    Sigma_inv_mu = solve(Sigma, mu)                  # more stable than inv
    w = Sigma_inv_mu / Sigma_inv_mu.sum()
    return w                                         # (N,)

def apply_weights(R_df: pd.DataFrame, w: np.ndarray, name: str):
    """Portfolio returns series R_tan_t = w' R_t on the given sample."""
    vals = R_df.values @ w
    return pd.Series(vals, index=R_df.index, name=name)

# -------------------------------------------------------------------
# Prepare the 25 portfolio EXCESS returns (your beme_portfolios already are)
# If you previously added a 'Tangency' column, drop it before computing w
# -------------------------------------------------------------------
R25 = beme_portfolios.copy()
if 'Tangency' in R25.columns:
    R25 = R25.drop(columns=['Tangency'])

# -------------------------------------------------------------------
# (a) Split into the two interleaved halves
# -------------------------------------------------------------------
R25_A, R25_B = split_interleaved(R25)

# -------------------------------------------------------------------
# (b) Estimate w on A, apply to B  (out-of-sample factor for B)
# -------------------------------------------------------------------
w_A = tangency_weights(R25_A)
Rtan_B = apply_weights(R25_B, w_A, name='R_tan_from_A')

# -------------------------------------------------------------------
# (c) Estimate w on B, apply to A  (out-of-sample factor for A)
# -------------------------------------------------------------------
w_B = tangency_weights(R25_B)
Rtan_A = apply_weights(R25_A, w_B, name='R_tan_from_B')

# Optional: collect the two OOS tangency factors in one DataFrame
tangency_oos = pd.concat([Rtan_A, Rtan_B], axis=1)

# -------------------------------------------------------------------
# Example: run regressions OOS using your existing function
#   (Use the factor that was built on the opposite half.)
#   Here, as an example, regress the 25 portfolios OOS; you can swap in
#   vw_industries (30 industries) or other test assets similarly.
# -------------------------------------------------------------------
# Regress on half A using factor built from B
regressions_A = estimate_regressions(
    x = tangency_oos[['R_tan_from_B']].loc[R25_A.index],
    y = R25.loc[R25_A.index]
)

# Regress on half B using factor built from A
regressions_B = estimate_regressions(
    x = tangency_oos[['R_tan_from_A']].loc[R25_B.index],
    y = R25.loc[R25_B.index]
)

# print weights estimated on A (used on B)
print("w_from_A:")
print(pd.Series(w_A, index=R25.columns).round(6))
print("sum(w_from_A) =", float(w_A.sum()))

# print weights estimated on B (used on A)
print("\nw_from_B:")
print(pd.Series(w_B, index=R25.columns).round(6))
print("sum(w_from_B) =", float(w_B.sum()))



w_from_A:
Small_Low    -0.388619
Small_2      -0.764958
Small_3      -0.603358
Small_4       1.247746
Small_High    1.611316
2_Low        -1.447250
2_2          -0.166898
2_3           0.596077
2_4          -0.385811
2_High       -0.259569
3_Low        -0.341718
3_2           0.690258
3_3           0.430325
3_4           1.292780
3_High        0.434401
4_Low         1.045363
4_2          -1.712044
4_3           0.681858
4_4          -0.159061
4_High       -0.880253
Big_Low       0.505837
Big_2         0.031639
Big_3         0.601409
Big_4        -1.218737
Big_High      0.159266
dtype: float64
sum(w_from_A) = 1.0

w_from_B:
Small_Low    -0.507262
Small_2      -0.859317
Small_3      -0.406292
Small_4       0.314666
Small_High    0.590495
2_Low        -0.541208
2_2           0.619613
2_3           0.167750
2_4           1.573709
2_High        0.781186
3_Low        -0.573779
3_2           1.109614
3_3           0.156230
3_4          -0.424105
3_High       -0.237054
4_Low         0.772666
4

In [23]:

Rtan_A = pd.Series(R25_A.values @ w_B, index=R25_A.index, name='R_tan_from_B')  # applied to A
Rtan_B = pd.Series(R25_B.values @ w_A, index=R25_B.index, name='R_tan_from_A')  # applied to B
Rtan_full_oos = pd.concat([Rtan_A, Rtan_B]).sort_index()
Rtan_full_oos.name = 'R_tan_full_oos'

print(Rtan_full_oos.head(100))

date
1926-07-01    -6.629462
1926-08-01    17.376519
1926-09-01     1.231406
1926-10-01    18.920053
1926-11-01    -1.759664
                ...    
1934-06-01     7.862975
1934-07-01    -9.327441
1934-08-01     2.286055
1934-09-01     5.907044
1934-10-01    -3.481768
Name: R_tan_full_oos, Length: 100, dtype: float64


In [25]:
import sys
print(sys.version)

3.13.5 (v3.13.5:6cb20a219a8, Jun 11 2025, 12:23:45) [Clang 16.0.0 (clang-1600.0.26.6)]
